In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive', force_remount= True)

Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
cd /content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet

/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet


In [ ]:
!pip install wandb

In [ ]:
import pandas as pd
import yaml
import argparse
import torch
from model import YNet

In [ ]:
%load_ext autoreload
%autoreload 2

#### Some hyperparameters and settings

In [ ]:
CONFIG_FILE_PATH = 'config/inD_longterm.yaml'  # yaml config file containing all the hyperparameters
EXPERIMENT_NAME = 'ind_longterm'  # arbitrary name for this experiment
DATASET_NAME = 'ind'

TRAIN_DATA_PATH = 'data/inD/train.pkl'
TRAIN_IMAGE_PATH = 'data/inD/train'
VAL_DATA_PATH = 'data/inD/test.pkl'
VAL_IMAGE_PATH = 'data/inD/test'
OBS_LEN = 5 # in timesteps
PRED_LEN = 30 # in timesteps
NUM_GOALS = 20 # K_e
NUM_TRAJ = 10 # K_a

BATCH_SIZE = 8

In [ ]:
pip install -r requirements.txt

     |████████████████████████████████| 67 kB 4.4 MB/s 
     |████████████████████████████████| 269 kB 18.0 MB/s 
     |████████████████████████████████| 753.2 MB 12 kB/s 
     |████████████████████████████████| 49.4 MB 1.2 MB/s 
     |████████████████████████████████| 25.9 MB 99.9 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 58 kB 8.2 MB/s 
     |████████████████████████████████| 23.2 MB 1.1 MB/s 
     |████████████████████████████████| 23.3 MB 1.1 MB/s 
     |████████████████████████████████| 23.3 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 1.2 MB/s 
     |████████████████████████████████| 22.1 MB 56 kB/s 
     |████████████████████████████████| 17.4 MB 65.7 MB/s 
     |████████████████████████████████| 17.3 MB 89.6 MB/s 
     |████████████████████████████████| 12.8 MB 75.5 MB/s 
     |████████████████████████████████| 12.7 MB 68.1 MB/s 
     |████████████████████████████████| 11.8 MB 46.9 MB/s 
     |█████████

#### Load config file and print hyperparameters

In [ ]:
with open(CONFIG_FILE_PATH) as file:
    params = yaml.load(file, Loader=yaml.FullLoader)
experiment_name = CONFIG_FILE_PATH.split('.yaml')[0].split('config/')[1]
params

{'CWS_params': {'ratio': 2, 'rot': True, 'sigma_factor': 6},
 'batch_size': 8,
 'decoder_channels': [64, 64, 64, 32, 32],
 'encoder_channels': [32, 32, 64, 64, 64],
 'kernlen': 31,
 'learning_rate': 0.0001,
 'loss_scale': 1000,
 'nsig': 4,
 'num_epochs': 300,
 'rel_threshold': 0.002,
 'resize': 0.33,
 'segmentation_model_fp': 'segmentation_models/inD_segmentation.pth',
 'semantic_classes': 6,
 'temperature': 1.8,
 'unfreeze': 100,
 'use_CWS': True,
 'use_TTST': True,
 'use_features_only': False,
 'viz_epoch': 10,
 'waypoints': [14, 29]}

#### Wandb INIT

#### Load preprocessed Data

In [ ]:
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)
!pip3 install pickle5
#df_train = pd.read_pickle(TRAIN_DATA_PATH)
#df_val = pd.read_pickle(VAL_DATA_PATH)

import pickle5 as pickle 
with open(TRAIN_DATA_PATH, "rb") as fh:
    df_train = pickle.load(fh)
with open(VAL_DATA_PATH, "rb") as fh1:
    df_val = pickle.load(fh1)

     |████████████████████████████████| 256 kB 7.3 MB/s 


In [ ]:
df_train.head()

,trackId,frame,x,y,sceneId,metaId
0,31,2217,25.07654,6.78323,07,0
1,31,2242,26.11484,7.72170,07,0
2,31,2267,27.05390,8.94723,07,0
3,31,2292,28.08326,10.18219,07,0
4,31,2317,29.08530,11.39276,07,0


#### Initiate model

In [ ]:
model = YNet(obs_len=OBS_LEN, pred_len=PRED_LEN, params=params)

/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.encoders.resnet.ResNetEncoder' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Conv2dReLU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/usr/local/lib/python3.7/dist-packages/torch/serialization.py:657: SourceChangeWarning: source code of class 'segmentation_models_pytorch.base.modules.Activation' has changed. you can retrieve the original 

#### Start training
Note, the Val ADE and FDE are without TTST and CWS to save time. Therefore, the numbers will be worse than the final values.

In [ ]:
import weights_and_biases as wandb
wandb.init_wandb(params.copy(), model.model)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
model.train(df_train, df_val, params, train_image_path=TRAIN_IMAGE_PATH, val_image_path=VAL_IMAGE_PATH, 
            experiment_name=EXPERIMENT_NAME, batch_size=BATCH_SIZE, num_goals=NUM_GOALS, num_traj=NUM_TRAJ, 
            device=None, dataset_name= 'ind')

Preprocess data


Prepare Dataset: 100%|██████████| 208/208 [00:00<00:00, 986.12it/s]
/content/drive/MyDrive/Human-Path-Prediction-master (1)/ynet/utils/dataloader.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(trajectories), meta, scene_list
Epoch:   0%|          | 0/300 [00:00<?, ?it/s]

Start training
208
Epoch 0: 
Val ADE: 30.448373794555664 
Val FDE: 19.351049423217773
Best Epoch 0: 
Val ADE: 30.448373794555664 
Val FDE: 19.351049423217773


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   0%|          | 1/300 [07:35<37:51:36, 455.84s/it]

Saved model to: saved_models/Ynet.pt
Epoch 1: 
Val ADE: 30.0434513092041 
Val FDE: 17.931123733520508
Best Epoch 1: 
Val ADE: 30.0434513092041 
Val FDE: 17.931123733520508


...

Epoch 19: 
Val ADE: 3.566770315170288 
Val FDE: 13.248457908630371
Epoch 20: 
Val ADE: 3.301231622695923 
Val FDE: 10.414738655090332
Best Epoch 20: 
Val ADE: 3.301231622695923 
Val FDE: 10.414738655090332


wandb: WARNING Saving files without folders. If you want to preserve sub directories pass base_path to wandb.save, i.e. wandb.save("/mnt/folder/file.h5", base_path="/mnt")
Epoch:   7%|▋         | 21/300 [2:39:02<35:12:30, 454.30s/it]

Saved model to: saved_models/Ynet.pt


Epoch:   7%|▋         | 22/300 [2:46:36<35:04:31, 454.22s/it]

Epoch 21: 
Val ADE: 4.333622932434082 
Val FDE: 13.820666313171387
